In [24]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import plotly.figure_factory as ff
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score

In [3]:
from google.colab import drive
drive.mount('/content/dive/', force_remount=True)

Mounted at /content/dive/


1. Загрузка файл HR.csv

In [5]:
df= pd.read_csv('/content/dive/MyDrive/Netologia/Статистика в Python/HR.csv')

2. Расчет основных статистик для переменных

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company
50%,0.66,0.72,4.00,200.00,3.00
mean,0.63,0.72,3.80,200.47,3.36
min,0.09,0.36,2.00,96.00,2.00
max,1.00,1.00,7.00,310.00,10.00
std,0.24,0.17,1.16,48.73,1.33


In [6]:
def get_standart_static_for_categorical_params(x):
  print('------------------------------------------------------------------')
  print(x.name)
  h = pd.Series(x)
  print('median:', h.median())
  print('mode:', h.mode())
  print('unique:', h.unique())
  print(h.value_counts())

In [7]:
def get_standart_static_for_continuous_quantities_params(x):
  print('------------------------------------------------------------------')
  print(x.name)
  h = pd.Series(x)
  print('min:', h.min())
  print('max:', h.max())
  print('range:', h.max() - h.min())
  print('disp:', round(h.var()))
  print('mean:', h.mean())
  print('median:', h.median())

  iqr = h.quantile(0.75) - h.quantile(0.25)
  print('iqr', iqr)
  lower_bound = h.quantile(0.25) - (1.5 * iqr)
  upper_bound = h.quantile(0.75) + (1.5 * iqr)
  outliers =  [x for x in h if x > upper_bound or x < lower_bound]
  print('outliers:', outliers)
  # print('unique:', h.unique())

In [27]:
# df.iloc[:,:5].describe().round(2).loc[['50%', 'mean', 'min', 'max', 'std']]
get_standart_static_for_continuous_quantities_params(df['satisfaction_level'])
get_standart_static_for_continuous_quantities_params(df['last_evaluation'])
get_standart_static_for_continuous_quantities_params(df['number_project'])
get_standart_static_for_continuous_quantities_params(df['average_montly_hours'])
get_standart_static_for_continuous_quantities_params(df['time_spend_company'])

------------------------------------------------------------------
satisfaction_level
min: 0.09
max: 1.0
range: 0.91
disp: 0
mean: 0.6296580768910016
median: 0.66
iqr 0.33999999999999997
outliers: []
------------------------------------------------------------------
last_evaluation
min: 0.36
max: 1.0
range: 0.64
disp: 0
mean: 0.7166825118839131
median: 0.72
iqr 0.29000000000000004
outliers: []
------------------------------------------------------------------
number_project
min: 2
max: 7
range: 5
disp: 1
mean: 3.802852139104328
median: 4.0
iqr 2.0
outliers: []
------------------------------------------------------------------
average_montly_hours
min: 96
max: 310
range: 214
disp: 2374
mean: 200.4735218080227
median: 200.0
iqr 86.0
outliers: []
------------------------------------------------------------------
time_spend_company
min: 2
max: 10
range: 8
disp: 2
mean: 3.3648569760653824
median: 3.0
iqr 1.0
outliers: [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 

In [34]:
get_standart_static_for_categorical_params(df['Work_accident'])
get_standart_static_for_categorical_params(df['left'])
get_standart_static_for_categorical_params(df['promotion_last_5years'])

------------------------------------------------------------------
Work_accident
median: 0.0
mode: 0    0
Name: Work_accident, dtype: int64
unique: [0 1]
0    10141
1     1850
Name: Work_accident, dtype: int64
------------------------------------------------------------------
left
median: 0.0
mode: 0    0
Name: left, dtype: int64
unique: [1 0]
0    10000
1     1991
Name: left, dtype: int64
------------------------------------------------------------------
promotion_last_5years
median: 0.0
mode: 0    0
Name: promotion_last_5years, dtype: int64
unique: [0 1]
0    11788
1      203
Name: promotion_last_5years, dtype: int64


In [8]:
df = df.drop_duplicates().reset_index(drop=True)
df.head(5)

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [9]:
df[df.duplicated() == True]

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,department,salary


3. Рассчитаем и визуализируем корреляционную матрицу для количественных переменных. Определим две самые скоррелированные и две наименее скоррелированные переменные

In [12]:
df.iloc[:,:5].corr().round(3)
df_corrs = df[['satisfaction_level', 'last_evaluation', 'number_project', 'average_montly_hours', 'time_spend_company']].corr()

fig = ff.create_annotated_heatmap(z=df_corrs.values, x=list(df_corrs.index),
                                  y=list(df_corrs.columns),
                                  annotation_text=df_corrs.round(3).values,
                                  showscale=True, colorscale='tropic')
fig.show()

4. Количество сотрудников

In [13]:
df.groupby('department').size()

department
IT              976
RandD           694
accounting      621
hr              601
management      436
marketing       673
product_mng     686
sales          3239
support        1821
technical      2244
dtype: int64

5. Распределение сотрудников по зарплатам

In [14]:
df.salary.value_counts()

low       5740
medium    5261
high       990
Name: salary, dtype: int64

6. Распределение сотрудников по зарплатам в каждом департаменте по отдельност

In [15]:
df.pivot_table(index='department', columns='salary', aggfunc='size')

salary,high,low,medium
department,,,
IT,71,476,429
RandD,47,322,325
accounting,63,296,262
hr,38,296,267
management,128,139,169
marketing,62,310,301
product_mng,52,343,291
sales,237,1553,1449
support,126,867,828


7. Проверим гипотезу, что сотрудники с высоким окладом проводят на работе больше времени, чем сотрудники с низким окладом

In [16]:
print('Среднее количество рабочих часов сотрудников c низким окладом', df[df.salary == 'low']['average_montly_hours'].mean())
print('Среднее количество рабочих часов сотрудников с высоким окладом', df[df.salary == 'high']['average_montly_hours'].mean())

Среднее количество рабочих часов сотрудников c низким окладом 200.4391986062718
Среднее количество рабочих часов сотрудников с высоким окладом 200.06767676767677


In [19]:
salary_low = df[df['salary'] == 'low']['average_montly_hours']
salary_high = df[df['salary'] == 'high']['average_montly_hours']
result = stats.ttest_ind(salary_low, salary_high, equal_var=False)

a = 0.05

if (result.pvalue < a):
    print('Отвергаем нулевую гипотезу')
else:
    print('Не отвергаем нулевую гипотезу')

Не отвергаем нулевую гипотезу


8. Рассчитаем показатели среди уволившихся и не уволившихся сотрудников (по отдельности)

In [20]:
workers = df[(df['left'] == 0)]
retireds =  df[(df['left'] == 1)]
indicators = pd.DataFrame()
rows = []

rows.append(
    {
    'Показатель': 'Доля получивших повышение',
    'Уволившиеся': round(retireds.promotion_last_5years.value_counts(normalize=True)[1], 3),
    'Действующие': round(workers.promotion_last_5years.value_counts(normalize=True)[1], 3)
    }
)

rows.append(
    {
    'Показатель': 'Средняя степень удовлетворенности',
    'Уволившиеся': round(retireds.satisfaction_level.mean(), 3),
    'Действующие': round(workers.satisfaction_level.mean(), 3)
    }
)

rows.append(
    {
    'Показатель': 'Среднее количество проектов',
    'Уволившиеся': round(retireds.number_project.mean(), 3),
    'Действующие': round(workers.number_project.mean(), 3)
    }
)

for row in rows:
    indicators = pd.concat([indicators, pd.DataFrame([row])])

indicators

,Показатель,Уволившиеся,Действующие
0,Доля получивших повышение,0.004,0.020
0,Средняя степень удовлетворенности,0.440,0.667
0,Среднее количество проектов,3.883,3.787


9. Разделим данные на тестовую и обучающую выборки. Построим модель LDA, предсказывающую уволился ли сотрудник на основе имеющихся факторов (кроме department и salary). Оценим качество модели на тестовой выборке

In [25]:
X = df.iloc[:,[0,1,2,3,4,5,7]]
y = df['left']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
model = LinearDiscriminantAnalysis()
model.fit(X_train, y_train)
model.predict(X_test)
model.score(X_test, y_test).round(2)
accuracy_score(y_test, model.predict(X_test)).round(2)
#Коэффициент детерминации - 0.83, а это означает, что модель получилась хорошая (коэффициент стремится к 1).

0.83